In [6]:
from pyspark.sql import SparkSession
print(type(spark))
csv = spark.read.csv("personalization_theory/ratings_Electronics.csv")

# csv2 = spark.read.csv("ratings_Video_Games.csv")
# csv = csv.union(csv2)
csv = csv.selectExpr("_c0 as user", "_c1 as item", "_c2 as rating", "_c3 as time")
distinct_val = csv.select('item').distinct()

<class 'pyspark.sql.session.SparkSession'>


In [7]:
from pyspark.sql.types import IntegerType

changedTypedf = csv.withColumn("time_int", csv["time"].cast(IntegerType()))

In [9]:
changedTypedf.approxQuantile("time_int", [0.5], 0.25)


[1403395200.0]

In [12]:
old = changedTypedf.filter(changedTypedf.time_int < 1403395200)

In [13]:
new = changedTypedf.filter(changedTypedf.time_int >= 1403395200)

In [2]:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, LongType

with_index = distinct_val.rdd.zipWithIndex()
distinct_val = with_index.map(lambda x: (Row(item = x[0].item, id =x[1])))
distinct_val = distinct_val.toDF()

In [24]:
distinct_val.take(2)
maxIndex = distinct_val.count()
print(maxIndex)

525967


In [28]:
from pyspark.sql.functions import rand 
csv = csv.orderBy(rand())

In [3]:
joined_df = csv.join(distinct_val, ['item'], "left")
rdd1 = joined_df.rdd

In [4]:
joined_df.take(10)

[Row(item='0439394422', user='A1TL721YECDIM8', rating='3.0', time='1389657600', id=2485),
 Row(item='0439394422', user='APDCEJMFDO2YT', rating='5.0', time='1285545600', id=2485),
 Row(item='7793224531', user='ACNP9LJ391F3L', rating='4.0', time='1287532800', id=0),
 Row(item='7793224531', user='A2QE3HV9GW264C', rating='5.0', time='1305936000', id=0),
 Row(item='7793224531', user='A4NLBWDS2USI', rating='5.0', time='1312502400', id=0),
 Row(item='7793224531', user='A2CZG9I6ERMOFA', rating='4.0', time='1284249600', id=0),
 Row(item='7793224531', user='A2CK2QW8OUZARL', rating='5.0', time='1397433600', id=0),
 Row(item='7793224531', user='AN3CN5M3U865J', rating='4.0', time='1329868800', id=0),
 Row(item='7793224531', user='AO8QRF4F2XXS1', rating='5.0', time='1265760000', id=0),
 Row(item='7793224531', user='AO4Q0RPYN3Y1N', rating='5.0', time='1321747200', id=0)]

In [31]:
rdd2 = rdd1.map(lambda line: (line.user, (line.id, line.rating))).groupByKey().cache()

In [32]:
rdd2.take(4)

[('A2HLOMXD15UQ5F', <pyspark.resultiterable.ResultIterable at 0x177e34c18>),
 ('A8QDWEWU0EZ4E', <pyspark.resultiterable.ResultIterable at 0x177e34c50>),
 ('AZT8EE93WXLJZ', <pyspark.resultiterable.ResultIterable at 0x177e34da0>),
 ('A3152KT8N27WTF', <pyspark.resultiterable.ResultIterable at 0x177e34828>)]

In [33]:
import pprint
print(list(rdd2.take(2)[1]))
pprint.pprint(list(rdd2.take(2)[1][1]))

['A8QDWEWU0EZ4E', <pyspark.resultiterable.ResultIterable object at 0x177e2ec88>]
[(2515, '5.0')]


In [9]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector

def sv_format(x):
    item = []
    rating = []
    for i in x:
        item.append(i[0])
        rating.append(int(float((i[1]))))
    points = zip(item, rating)
    sorted_points = sorted(points)
    new_xs = [point[0] for point in sorted_points]
    new_ys = [point[1] for point in sorted_points]

    return new_xs, new_ys

sparseVectorData = rdd2.map(lambda a :(a[0], sv_format(a[1])))

In [34]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector

def sv_format(x):
    item = {}
    for i in x:
        item[i[0]] = int(float((i[1])))
    return item

sparseVectorData = rdd2.map(lambda a :(a[0], sv_format(a[1])))

In [35]:
sparseVectorData

PythonRDD[248] at RDD at PythonRDD.scala:48

In [36]:
myfile = sparseVectorData.collect()

In [47]:
myfile1 = myfile[:int(len(myfile)/10)]

In [52]:
myfile.rdd

('AKWMVB3BIPVJD', {111: 1, 13526: 5, 132171: 5, 196655: 5, 283926: 5})

In [48]:
import pandas as pd
my_df = pd.DataFrame(myfile1)
my_df.to_csv('1-10.csv', index=False, header=False)

In [17]:
"""
this block of code generates random .5% of users with older ratings
"""
from pyspark.sql import Row
from pyspark.sql.functions import rand 
import pandas as pd

distinct_val = old.select('item').distinct()
with_index = distinct_val.rdd.zipWithIndex()
distinct_val = with_index.map(lambda x: (Row(item = x[0].item, id =x[1])))
distinct_val = distinct_val.toDF()
old = old.orderBy(rand())
joined_df = old.join(distinct_val, ['item'], "left")
rdd1 = joined_df.rdd
rdd2 = rdd1.map(lambda line: (line.user, (line.id, line.rating))).groupByKey().cache()

def sv_format(x):
    item = {}
    for i in x:
        item[i[0]] = int(float((i[1])))
    return item

sparseVectorData = rdd2.map(lambda a :(a[0], sv_format(a[1])))
myfile = sparseVectorData.collect()
myfile1 = myfile[:int(len(myfile)/200)]
my_df = pd.DataFrame(myfile1)
my_df.to_csv('old.csv', index=False, header=False)

In [18]:
"""
this block of code generates random .5% of users with newer ratings
"""
distinct_val = new.select('item').distinct()
with_index = distinct_val.rdd.zipWithIndex()
distinct_val = with_index.map(lambda x: (Row(item = x[0].item, id =x[1])))
distinct_val = distinct_val.toDF()
new = new.orderBy(rand())
joined_df = new.join(distinct_val, ['item'], "left")
rdd1 = joined_df.rdd
rdd2 = rdd1.map(lambda line: (line.user, (line.id, line.rating))).groupByKey().cache()

def sv_format(x):
    item = {}
    for i in x:
        item[i[0]] = int(float((i[1])))
    return item

sparseVectorData = rdd2.map(lambda a :(a[0], sv_format(a[1])))
myfile = sparseVectorData.collect()
myfile1 = myfile[:int(len(myfile)/200)]
my_df = pd.DataFrame(myfile1)
my_df.to_csv('new.csv', index=False, header=False)